# JOIN GROUPS

In [1]:
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from sklearn.preprocessing import normalize

In [2]:

def leer_data():
    outfile='../RECURSOS/data.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data = data_original[['client_id','date','año','mes','dia','hora','mccg','quantity','amount_sol']]
data.head(5) # Data de la consulta

,client_id,date,año,mes,dia,hora,mccg,quantity,amount_sol
0,NNeQwQy9MAQ=,2016-07-15 22:23:25,2016,7,15,22,2,1,100.0
1,qFRoPHPOc/I=,2016-06-21 21:30:55,2016,6,21,21,2,1,20.0
2,qFRoPHPOc/I=,2017-02-01 01:29:59,2017,2,1,1,2,1,100.0
3,qFRoPHPOc/I=,2017-03-09 02:26:05,2017,3,9,2,2,1,50.0
4,he9pUcExpTY=,2016-10-01 06:45:48,2016,10,1,6,2,1,100.0


In [3]:
mccgs = ((data.groupby(['mccg'], as_index=False, sort=True)).count())['mccg']
#mccgs,mccgs[3],len(mccgs)

# RESULTADOS

## Unimos los resultados por cada mccg

Juntamos los datos originales con sus etiquetas de cluster individuale y cluster colectivo correspondientes.

<div>Utulizamos solo 2 variables para el ahorro de memoria. </div>

In [4]:

path='./RESULTADOS' 

file="%s/U" %(path) 
footprint="%s.footprint" %(file)
individual_clusters="%s.individual_footprint.clusters" %(file)
individual_labels="%s.individual_footprint.labels" %(file)   
collective_clusters="%s.collective_footprint.clusters" %(file)
collective_labels="%s.collective_footprint.labels" %(file)


In [5]:
a = pd.read_csv(individual_labels, sep=";", header=0, index_col=False, dtype={'week': str,'year': str})   ## read file
b = pd.read_csv(collective_labels, sep=";", header=0, index_col=False)   ## read file

In [6]:
a = pd.merge(a, b, on=['customer_tag', 'individual_cluster'])
del b
a = a.iloc[:, :(-1*4*7*len(mccgs))]

In [7]:
b = pd.read_csv(footprint, sep=",", header=0, dtype={'week': str,'year': str})   ## read file
b.rename(columns={'customer_id':'customer_tag'}, inplace=True)  

In [8]:
a = pd.merge(b, a, on=['customer_tag','year', 'week'])
del b
b = a.iloc[:, :(-1*4*7*len(mccgs)-1)]

In [9]:
a = pd.concat([b, a[['collective_cluster']]], axis=1)
del b
a[['year','week','profile_id','size','individual_cluster','collective_cluster']] = a[['year','week','profile_id','size','individual_cluster','collective_cluster']]

In [10]:
a.head(5)

,customer_tag,year,week,profile_id,mccg,turn,size,m2t0d0,m2t0d1,m2t0d2,...,m5t2d6,m5t3d0,m5t3d1,m5t3d2,m5t3d3,m5t3d4,m5t3d5,m5t3d6,individual_cluster,collective_cluster
0,+9yJoBsAES4=,2016,24,0,3,2,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,10
1,+EUZmfQX5bA=,2017,10,0,2,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,45
2,+Ilmh+mr2WM=,2017,19,0,2,3,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,32
3,+Ilmh+mr2WM=,2017,21,1,2,0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,65
4,+Ilmh+mr2WM=,2017,01,2,2,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,65


In [11]:
path_res='%s/Results.csv' %(path)
a.to_csv(path_res,index=False)
print('Done')

Done
